In [0]:
%sql
USE CATALOG content

In [0]:
%sql
SELECT current_catalog()

In [0]:
from pyspark.sql import functions as f
df = spark.table("content.target.gold_account_master_view")
df = df.select([c for c in df.columns if c not in {'account_name','verificationConfidence','isVerified'}])
df.display()

In [0]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol = "accountAgeCategory", outputCol = "accountAgeCategory_index")
df_ = df.dropna()
df_indexed = indexer.fit(df_).transform(df_)



In [0]:
#from pyspark.sql import functions as F, col
#df_indexed.display()
df_indexed = df_indexed.withColumn("potentialInfluencer", f.when(f.col("potentialInfluencer") == True, 1).otherwise(0))
#df_indexed.dtypes

In [0]:
df_indexed = df_indexed.drop("accountAgeCategory")

In [0]:
df_indexed.display()

In [0]:
df_indexed_ = df_indexed.toPandas()
X = df_indexed_.drop("potentialInfluencer", axis = 1)
Y = df_indexed_["potentialInfluencer"]

X

In [0]:

X

In [0]:
import mlflow
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Automatyczne logowanie parametrów i metryk
#mlflow.autolog()

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

with mlflow.start_run(run_name="Bot_Detection_v1"):
    model = XGBClassifier(n_estimators=5, max_depth=2)
    model.fit(X_train, y_train)
    
   # predictions = model.predict(X_test)
   # accuracy = accuracy_score(y_test, predictions)
   # print(f"Model Accuracy: {accuracy:.2f}")

In [0]:
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Model Accuracy: {accuracy:.2f}")
